# 9. Dueling DQN
- DQN 算法敲开了深度强化学习的大门，但是作为先驱性的工作，其本身存在着一些问题以及一些可以改进的地方。于是，在 DQN 之后，学术界涌现出了非常多的改进算法...

其中有两个非常著名的算法：**Double DQN 和 Dueling DQN**
本节学习 **Dueling DQN** --- **https://arxiv.org/abs/1511.06581** 

(更多、更详细的 DQN 改进方法，见 Rainbow 模型的论文及其引用文献:[0]Rainbow: Combining improvements in deep reinforcement learning)

## 9.1 Dueling DQN 原理
根据前期的学习可知:在同一状态下,**所有动作**的动作价值的期望就是**该状态**的状态价值
那么完全可以将 **状态价值** 剥离出来
将 **状态动作价值函数Q** 减去 **状态价值函数V** 的结果定义为 **优势函数A** :$A(s,a)=Q(s,a)-V(s)$
该函数可以独自表达各个动作在同一状态下的 **差异**，即衡量在状态 s 下选择动作 a 相对于平均水平的好坏

实际上原始的DQN算法Q值也有大小之分,那么为何要多此一举定义一个独自表达差异的函数呢?
本质上是要提升网络学习效率和泛化能力:
- 不是所有状态都需要区分动作(核心)：如在 Atari 游戏中，角色正处于安全区，向左/向右/跳跃都不会立即影响得分，但原始 DQN 仍要为每个动作分别拟合 Q 值
- 更结构化，对动作多差异小的状态更有优势：分离状态与差异,根据实际情况有选择的关注(当智能体前面没有车时，车辆自身动作并没有太大差异，此时智能体更关注状态价值；而当智能体前面有车时（智能体需要超车），智能体开始关注不同动作优势值的差异)

![状态价值和优势值的简单例子](Illustrations/状态价值和优势值的简单例子.png)


在 Dueling DQN 中，Q 网络被重建为：
$$Q_{\eta,\alpha,\beta}(s,a)=V_{\eta,\alpha}(s)+A_{\eta,\beta}(s,a)$$
#### 参数说明:
$\eta$(整个函数近似器（神经网络）的共享参数)：在 Dueling DQN 网络中，该参数是整个函数近似器（神经网络）的共享参数，用于前面几层提取共享状态的表示
$\alpha$：专用于拟合状态价值函数的网络分支的参数，而该网络从共享特征中抽取更专注于“状态本身”的信息
$\beta$：专用于拟合优势函数的网络分支的参数，该网络主要用于捕捉特定动作的相对优势

以下为网络结构对比图：
![DQN与Dueling%20DQN网络结构](Illustrations/DQN与Dueling%20DQN网络结构.jpg)
- 同时如此设计Q网络，可以使每一次更新时，函数都会被更新，这也会影响到其他动作的Q值(传统的 DQN 只会更新某个动作的值) 

因此，Dueling DQN 能够更加频繁、准确地学习状态价值函数

#### 不唯一性的问题：
##### 来源
$$Q(s,a)=V(s)+A(s,a)=
\begin{pmatrix}
V(s)+C(s)
\end{pmatrix}+
\begin{pmatrix}
A(s,a)-C(s)
\end{pmatrix}$$

- 其中$C(s)$是任意与动作无关的函数（只依赖于状态）

可见分解并不是唯一，存在无限多组 V 和 A 的组合，使得加起来等于相同的 Q 这就导致了训练的不稳定性

##### 解决
为了消除这种不唯一性，Dueling DQN通常会使用一些归一化策略，使优势函数满足一定**约束**，最经典的有：
1. 减去最大优势：强制最优动作的优势函数的实际输出为 0 
$$Q(s,a)=V(s)+\left(A(s,a)-\max_{a^\prime}A(s,a^\prime)\right)$$
此时有$V(s)=\max_{a}Q(s,a)$

2. 减去优势函数的均值：优势函数去中心化，保证所有动作优势的平均值为 0
$$Q(s,a)=V(s)+\left(A(s,a)-\frac{1}{|\mathcal{A}|}\sum_{a^{\prime}}A(s,a^{\prime})\right)$$
此时有$V(s)=\frac{1}{|\mathcal{A}|}\sum_{a^{\prime}}Q(s,a^{\prime})$

##### 常使用第2种方法,更稳定
但会导致该结构形式下的 Q 值不再严格满足 Bellman 最优方程，以下为数学直观表达:
$$A(s,a_1)=4,\quad A(s,a_2)=2$$
$$\frac{1}{|\mathcal{A}|}\sum_{a^{\prime}}Q(s,a^{\prime})=(4+2)/2=3$$
$$\begin{gathered}
Q(s,a_1)=10+(4-3)=11 \\
Q(s,a_2)=10+(2-3)=9
\end{gathered}$$
| Q值       | 原始方式 | 归一化后 | 差异   |
|----------| ---- | ---- | ---- |
| $Q(s,a_1)$ | 14   | 11   | ⬇️ 3 |
| $Q(s,a_2)$ | 12   | 9    | ⬇️ 3 |
| $\max Q$ | 14   | 11   | ⬇️ 3 |
- 网络学到的 Q 实际不是 Bellman 方程中的 Q 值，而是“减了常数”的版本，偏离真实更新目标

尽管在训练后期这种影响，不会磨灭动作之间的本质差异，但是RL想接近精确最优，需要结构满足 Bellman 最优性
复杂任务下，不精确会迅速放大误差

## 9.2 Dueling DQN 代码实现

导入相关库：

In [5]:
from utils.replay_buffer import ReplayBuffer
from utils.train import train_on_policy_agent, train_off_policy_agent
from utils.advantage import compute_advantage
from utils.smoothing import moving_average